## Glossário de Termos e Dicionário de Dados


### 1.  Métricas de Qualidade de Dados (ETL)

| Termo | Definição | Base de Cálculo/Contexto |
| :--- | :--- | :--- |
| **Completude de CNPJ** | Percentual de registros com CNPJ válido (14 dígitos) após limpeza. | `(df['cnpj_instituicao'] != '').mean() * 100` |
| **Taxa de Inversão Corrigida** | Percentual de registros onde foi detectada e corrigida inversão CNPJ/Nome. | Métrica de qualidade do ETL para dados históricos. |
| **Encoding Detectado** | Codificação de caracteres identificada para leitura correta do arquivo. | `utf-8-sig, latin-1, iso-8859-1` (tentativas em ordem) |
| **Taxa de Unicidade de Pedidos** | Percentual de registros com identificadores únicos após processamento. | (ids_únicos / total_registros) * 100 (263.259/263.562 = 99.9%) |

---

### 2. Termos Técnicos do ETL

| Termo| Descrição | Uso no Projeto |
| :--- | :--- | :--- |
| **Pipeline de Tolerância a Falhas** | ETL que continua processamento mesmo com erro em arquivos individuais. | Processa 2020-2022 mesmo se um ano falhar |
| **Limpeza Regional Brasileira** | Rotina específica para formatos monetários BR (R$, vírgula decimal, ponto milhar). | Conversão robusta de preco_unitario |
| **Colunas de Compatibilidade** | Campos adicionados para garantir schema uniforme entre dados antigos e novos. | capacidade, unidade_medida com valores padrão |
| **Heurística Alfanumérica** | Lógica baseada em contagem de letras vs. dígitos para identificar nomes vs. CNPJs. | Correção automática de colunas trocadas |
| **Geração de Hash ID** | Criação de identificador único combinando múltiplos atributos do pedido.	| hash(cnpj_instituicao + codigo_br + data_compra + ...) |

---

### 3. Estruturas de Dados Criadas

| Termo | Descrição | Propósito |
| :--- | :--- | :--- |
| **Schema de Consolidação** | Ordem padronizada de colunas para unificação de datasets. | Garante compatibilidade no Data Warehouse |
| **Mapa de Renomeação** | Dicionário explícito de mapeamento de colunas antigas para novas. | Trata caracteres invisíveis e inconsistências |
| **DataFrame de Consolidação** | Dataset final unificado de múltiplos anos/arquivos. |	Base para análises temporais 2020-2022 |
| **Star Schema** | Modelo dimensional com tabela fato central e dimensões relacionadas. | Otimização para análises multidimensionais |
| **Chaves Surrogate** | Identificadores artificiais com prefixos mnemônicos (Ins, Pro, For, Fab). | Garantia de unicidade e performance |

---

### 4. Métricas de Performance do ETL

| Termo | Definição | Importância |
| :--- | :--- | :--- |
| **Taxa de Sucesso por Arquivo** |	Percentual de arquivos processados com sucesso. | Indicador de robustez do pipeline |
| **Registros Processados por Ano** | Volume de dados limpos gerado por período. | Métrica de escala e cobertura temporal |
| **Tempo de Processamento** | Duração do ETL desde leitura até consolidação. |	Otimização de performance|
| **Redução de Redundância** | Percentual de economia em armazenamento de dados descritivos. | 87.8% redução (32.127 vs 1.3M+ instâncias) |

---

### 5. Termos do Dataset e Nomenclaturas

| Termo | Descrição | Uso no Projeto |
| :--- | :--- | :--- |
| **Código BR** | Código oficial de identificação do produto usado no OpenDataSUS (ou fonte oficial de origem). | Chave primária limpa e padronizada da Dimensão Produto. |
| **CATMAT** | Classificação Comum de Materiais (do SISMAD). | Usada em *tooltips* para detalhar a descrição do produto. |
| **Mini Fato Radar** | Tabela auxiliar criada no ETL para armazenar as colunas pré-calculadas: **PMP Mediano**, **Z-Score** e **Economia por Linha**. | Usado para alimentar as métricas do Dashboard de Risco/Oportunidade. |
| **Drill-Through** | Funcionalidade do Power BI que permite ao usuário clicar em um ponto do visual e ser levado à página de investigação detalhada. | Focado na investigação do Fornecedor, Fabricante e Modalidade de Compra. |
| **INDEFINIDO** | Valor de *fallback* (substituto) para categorias inválidas ou nulas. | Garante que a Dimensão `Tipo_Compra` não contenha valores nulos. |


---

### 6. Técnicas de Engenharia de Dados (ETL)

| Técnica | Descrição e Justificativa Técnica | Aplicação no Projeto |
| :--- | :--- | :--- |
| **Data Wrangling** | Termo profissional para a limpeza, padronização e transformação de dados brutos em um formato consistente e utilizável. | Aplicação de rotinas robustas de limpeza regional (moeda, data). |
| **Heurística de Dados** | Lógica baseada em regras (ex: contagem de caracteres alfabéticos vs. dígitos) para inferir e corrigir a qualidade de dados. | Usada para corrigir a **inversão de colunas** CNPJ/Nome em datasets históricos. |
| **dayfirst=True** | Parâmetro no `pd.to_datetime` que força a interpretação do formato de data como Dia/Mês/Ano. | Essencial para o tratamento de formatos de data regionais brasileiros. |
| **errors='coerce'** | Parâmetro no `pd.to_numeric` que instrui o Pandas a transformar qualquer valor que não possa ser convertido para numérico em `NaN`. | Essencial para o tratamento de valores sujos em colunas monetárias, preparando-os para o `fillna(0)`. |
| **Modelagem Dimensional** | Técnica de modelagem de dados que separa dados transacionais (fato) de dados descritivos (dimensões). | Implementação de Star Schema para otimização analítica |

---

### 7. MODELAGEM DIMENSIONAL

| Termo | Descrição | Contexto no Projeto |
| :--- | :--- | :--- |
| **Tabela Fato** | Tabela central com métricas transacionais e chaves para dimensões. | 263.562 registros de compras com preços e quantidades |
| **Dimensão Instituição** | Tabela com dados únicos de compradores (hospitais, municípios). | 1.132 registros únicos com dados institucionais |
| **Dimensão Produto** | Tabela com catálogo completo de medicamentos e insumos. | 21.257 registros únicos com descrições CATMAT |
| **Dimensão Fornecedor** | Tabela com cadastro de empresas fornecedoras. | 4.795 registros únicos com CNPJs e nomes |
| **Dimensão Fabricante** | Tabela com laboratórios e fabricantes dos produtos.	| 4.380 registros únicos da indústria farmacêutica |
| **Dimensão Tempo** | Tabela com hierarquia temporal para análises sazonais. | 1.563 datas únicas com ano, mês, trimestre |
| **Integridade Referencial** | Garantia de que todas as chaves estrangeiras têm correspondência nas dimensões. | 100% das relações fato-dimensão validadas |

---

### 8. INDICADORES DE GESTÃO

| Termo | Definição | Aplicação |
| :--- | :--- | :--- |
| **PMP (Preço Médio Ponderado)** | Métrica de referência para benchmark de preços entre instituições. | Base para cálculo de oportunidades de economia. |
| **Índice de Priorização** | Métrica composta para rankear oportunidades de compra. | Combina Z-Score risco + valor demanda |
| **Risco de Intermitência** | Indicador de estabilidade/instabilidade da demanda por produto. | Base para gestão de estoques e prevenção de desabastecimento |
| **Concentração de Fornecedor** | Percentual de dependência de fornecedor único por produto/região. | Alerta para risco de desabastecimento |
| **Z-Score de Risco** | Medida estatística de desvio do preço em relação à média. | Identificação de outliers de preço para auditoria |




		
		


